# Chapter 24: Parser Combinators

In [1]:
:!stack install trifecta

In [2]:
import Text.Trifecta

In [5]:
stop :: Parser a
stop = unexpected "stop"

In [10]:
one :: Parser Char
one = char '1'

In [14]:
one' :: Parser a
one' = one >> stop

In [30]:
oneTwo = char '1' >> char '2'

oneTwo' = oneTwo >> stop

testParse' :: String -> Parser Char -> Result Char
testParse' s p = parseString p mempty s

testParse :: Parser Char -> Result Char
testParse = testParse' "123"

In [17]:
:t parseString

parseString :: forall a. Parser a -> Delta -> String -> Result a

In [21]:
testParse stop

Failure (ErrInfo {_errDoc = (interactive):1:1: error: unexpected
    stop
123<EOF> 
^        , _errDeltas = [Columns 0 0]})

In [22]:
testParse one

Success '1'

In [23]:
testParse one'

Failure (ErrInfo {_errDoc = (interactive):1:2: error: unexpected
    stop
123<EOF> 
 ^       , _errDeltas = [Columns 1 1]})

In [24]:
testParse oneTwo

Success '2'

In [25]:
testParse oneTwo'

Failure (ErrInfo {_errDoc = (interactive):1:3: error: unexpected
    stop
123<EOF> 
  ^      , _errDeltas = [Columns 2 2]})

In [36]:
:t eof

eof :: forall (m :: * -> *). Parsing m => m ()

In [38]:
one'' = one <* eof
testParse one''

Failure (ErrInfo {_errDoc = (interactive):1:2: error: expected: end of input
123<EOF> 
 ^       , _errDeltas = [Columns 1 1]})

In [39]:
testParse' "1" one''

Success '1'

In [40]:
oneTwo'' = oneTwo <* eof
testParse oneTwo''

Failure (ErrInfo {_errDoc = (interactive):1:3: error: expected: end of input
123<EOF> 
  ^      , _errDeltas = [Columns 2 2]})

In [41]:
testParse' "12" oneTwo''

Success '2'

In [42]:
:t string

string :: forall (m :: * -> *). CharParsing m => String -> m String

In [46]:
parseString (string "123") mempty "123"

Success "123"

In [48]:
parseString (string "1") mempty "123"

Success "1"

In [50]:
parseString (string "12") mempty "123"

Success "12"

In [52]:
parseString (string "123") mempty "123"

Success "123"

In [54]:
string' :: String -> Parser String
string' = traverse char

In [60]:
parseString (string' "123") mempty "123"

Success "123"

In [66]:
import Control.Applicative

In [67]:
:info Alternative

class Applicative f => Alternative (f :: * -> *) where
  empty :: f a
  (<|>) :: f a -> f a -> f a
  some :: f a -> f [a]
  many :: f a -> f [a]
  {-# MINIMAL empty, (<|>) #-}
  	-- Defined in ‘GHC.Base’
instance Alternative ZipList -- Defined in ‘Control.Applicative’
instance GHC.Base.MonadPlus m => Alternative (WrappedMonad m) -- Defined in ‘Control.Applicative’
instance (Control.Arrow.ArrowZero a, Control.Arrow.ArrowPlus a) => Alternative (WrappedArrow a b) -- Defined in ‘Control.Applicative’
instance Alternative Result -- Defined in ‘Text.Trifecta.Result’
instance Alternative m => Alternative (Unspaced m) -- Defined in ‘Text.Parser.Token’
instance Alternative m => Alternative (Unlined m) -- Defined in ‘Text.Parser.Token’
instance Alternative m => Alternative (Unhighlighted m) -- Defined in ‘Text.Parser.Token’
instance Alternative Parser -- Defined in ‘Text.Trifecta.Parser’
instance Alternative [] -- Defined in ‘GHC.Base’
instance Alternative Maybe -- Defined in ‘GHC.Base’
instance Alternative IO -- Defined in ‘GHC.Base’
instance [safe] Control.Monad.Trans.Error.Error e => Alternative (Either e) -- Defined in ‘Control.Monad.Trans.Error’

In [69]:
:info Parser

newtype Parser a
  = Parser {unparser :: forall r.
                        (a -> Err -> Text.Trifecta.Util.It.It Rope r)
                        -> (Err -> Text.Trifecta.Util.It.It Rope r)
                        -> (a -> Data.Set.Internal.Set String -> Text.Trifecta.Delta.Delta -> Data.ByteString.Internal.ByteString -> Text.Trifecta.Util.It.It Rope r)
                        -> (ErrInfo -> Text.Trifecta.Util.It.It Rope r)
                        -> Text.Trifecta.Delta.Delta
                        -> Data.ByteString.Internal.ByteString
                        -> Text.Trifecta.Util.It.It Rope r}
  	-- Defined in ‘Text.Trifecta.Parser’
instance Functor Parser -- Defined in ‘Text.Trifecta.Parser’
instance (Semigroup a, Monoid a) => Monoid (Parser a) -- Defined in ‘Text.Trifecta.Parser’
instance Semigroup a => Semigroup (Parser a) -- Defined in ‘Text.Trifecta.Parser’
instance Alternative Parser -- Defined in ‘Text.Trifecta.Parser’
instance Applicative Parser -- Defined in ‘Text.Trifecta.Parser’
instance Monad Parser -- Defined in ‘Text.Trifecta.Parser’
instance CharParsing Parser -- Defined in ‘Text.Trifecta.Parser’
instance DeltaParsing Parser -- Defined in ‘Text.Trifecta.Parser’
instance Errable Parser -- Defined in ‘Text.Trifecta.Parser’
instance Parsing Parser -- Defined in ‘Text.Trifecta.Parser’
instance TokenParsing Parser -- Defined in ‘Text.Trifecta.Parser’

In [72]:
parseString (stop <|> one) mempty "123"

Success '1'

In [79]:
parseString (one <|> stop) mempty "123"

Success '1'

In [86]:
parser = (string "123" <* eof) <|> (string "12" <* eof) <|> (string "1" <* eof)

In [87]:
parseString parser mempty "123"

Success "123"

In [88]:
parseString parser mempty "12"

Success "12"

In [90]:
parseString parser mempty "1"

Success "1"

In [92]:
parseString parser mempty "1234"

Failure (ErrInfo {_errDoc = (interactive):1:4: error: expected: end of input
1234<EOF> 
   ^      , _errDeltas = [Columns 3 3]})

In [94]:
parseString parser mempty "2"

Failure (ErrInfo {_errDoc = (interactive):1:1: error: expected: "1",
    "12", "123"
2<EOF> 
^      , _errDeltas = [Columns 0 0]})